In [1]:
!pip install pandas

In [3]:
import pandas as pd
import requests
from datetime import datetime
import zipfile
import io

town = "Wimbourne"

class CompaniesHouseBulkData:
    """
    Extract company data from Companies House bulk CSV download
    """

    def __init__(self):
        self.base_url = "http://download.companieshouse.gov.uk"
        self.data = None

    def download_bulk_data(self, year=None, month=None):
        """
        Download the latest bulk company data from Companies House

        Args:
            year: Year (defaults to current year)
            month: Month (defaults to current month)

        Returns:
            Boolean indicating success
        """
        if year is None or month is None:
            now = datetime.now()
            year = now.year
            month = now.month

        # Format the URL
        date_str = f"{year}-{month:02d}-01"
        url = f"{self.base_url}/BasicCompanyDataAsOneFile-{date_str}.zip"

        print(f"Downloading from: {url}")
        print("This may take a few minutes (file is ~400MB)...")

        try:
            # Download the ZIP file
            response = requests.get(url, stream=True)
            response.raise_for_status()

            # Extract and read the CSV
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                csv_filename = z.namelist()[0]
                print(f"Extracting {csv_filename}...")

                with z.open(csv_filename) as f:
                    # Read CSV with proper encoding
                    self.data = pd.read_csv(f, encoding='utf-8', low_memory=False)

            print(f"Successfully loaded {len(self.data):,} companies!")
            return True

        except requests.exceptions.HTTPError as e:
            print(f"Error downloading file: {e}")
            print("The file for this month may not be available yet.")
            print("Try the previous month or check: http://download.companieshouse.gov.uk/en_output.html")
            return False
        except Exception as e:
            print(f"Error: {e}")
            return False

    def filter_by_town(self, town_name):
        """
        Filter companies by town in their registered address

        Args:
            town_name: Name of the town to filter by (case-insensitive)

        Returns:
            DataFrame of matching companies
        """
        if self.data is None:
            print("No data loaded. Please download bulk data first.")
            return None

        # The CSV has a 'RegAddress.PostTown' column for town
        town_col = 'RegAddress.PostTown'

        if town_col not in self.data.columns:
            print(f"Column {town_col} not found. Available columns:")
            print(self.data.columns.tolist())
            return None

        # Filter (case-insensitive)
        mask = self.data[town_col].str.contains(town_name, case=False, na=False)
        filtered = self.data[mask].copy()

        print(f"Found {len(filtered):,} companies in {town_name}")
        return filtered

    def get_company_summary(self, df):
        """
        Get a clean summary of company data

        Args:
            df: DataFrame of companies

        Returns:
            DataFrame with selected columns
        """
        # Select relevant columns
        columns_to_keep = [
            'CompanyName',
            'CompanyNumber',
            'CompanyCategory',
            'CompanyStatus',
            'RegAddress.AddressLine1',
            'RegAddress.AddressLine2',
            'RegAddress.PostTown',
            'RegAddress.County',
            'RegAddress.PostCode',
            'IncorporationDate',
            'SICCode.SicText_1'
        ]

        # Only keep columns that exist
        available_cols = [col for col in columns_to_keep if col in df.columns]

        return df[available_cols]

    def export_to_csv(self, df, filename):
        """
        Export filtered data to CSV

        Args:
            df: DataFrame to export
            filename: Output filename
        """
        df.to_csv(filename, index=False)
        print(f"Data exported to {filename}")


def main():
    """
    Example usage
    """
    # Initialize
    bulk = CompaniesHouseBulkData()

    # Download the latest bulk data
    print("=" * 60)
    print("STEP 1: Downloading Bulk Company Data")
    print("=" * 60)

    success = bulk.download_bulk_data()

    if not success:
        print("\nTrying previous month...")
        now = datetime.now()
        prev_month = now.month - 1 if now.month > 1 else 12
        prev_year = now.year if now.month > 1 else now.year - 1
        success = bulk.download_bulk_data(year=prev_year, month=prev_month)

    if not success:
        print("Could not download data. Please check the Companies House website.")
        return

    # Filter by town
    print("\n" + "=" * 60)
    print("STEP 2: Filtering by Town")
    print("=" * 60)

    # Example: Find companies in town name
    #town = "Wimbourne"
    companies = bulk.filter_by_town(town)

    if companies is not None and len(companies) > 0:
        # Get summary
        summary = bulk.get_company_summary(companies)

        # Display first 10 results
        print(f"\nFirst 10 companies in {town}:")
        print("-" * 60)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        print(summary.head(10))

        # Export to CSV
        print("\n" + "=" * 60)
        print("STEP 3: Exporting Results")
        print("=" * 60)

        output_file = f"companies_in_{town.lower().replace(' ', '_')}.csv"
        bulk.export_to_csv(summary, output_file)

        # Show statistics
        print("\n" + "=" * 60)
        print("STATISTICS")
        print("=" * 60)
        print(f"Total companies found: {len(companies):,}")
        print(f"\nCompany Status breakdown:")
        print(companies['CompanyStatus'].value_counts())

        print(f"\nTop 5 Company Types:")
        print(companies['CompanyCategory'].value_counts().head())


if __name__ == "__main__":
    main()

STEP 1: Downloading Bulk Company Data
This may take a few minutes (file is ~400MB)...
Extracting BasicCompanyDataAsOneFile-2025-11-01.csv...
Successfully loaded 5,681,537 companies!

STEP 2: Filtering by Town
Found 18 companies in Wimbourne

First 10 companies in Wimbourne:
------------------------------------------------------------
                                          CompanyName  \
489333           ATHELSTAN COURT (MANAGEMENT) LIMITED   
725420   BLENHEIM (BRANKSOME PARK) MANAGEMENT LIMITED   
794052             BRAMLEY COURT (MANAGEMENT) LIMITED   
830449                            BROAD POWER LIMITED   
1069316                      CHRISTCHURCH OPTICAL LTD   
1978969                    GADMINSTER ESTATES LIMITED   
2072031                      GLOBAL PHARMA QA LIMITED   
2151304           GREENOAKS LANDSCAPING & ROOFING LTD   
2392600                        HOWARDS PROPERTIES LTD   
2874324                       KYNANCE ASSOCIATES LTD.   

                 CompanyCategory Com

In [5]:
#Filters with postcode

import pandas as pd
import requests
from datetime import datetime
import zipfile
import io

class CompaniesHouseBulkData:
    """
    Extract company data from Companies House bulk CSV download
    """

    def __init__(self):
        self.base_url = "http://download.companieshouse.gov.uk"
        self.data = None

    def download_bulk_data(self, year=None, month=None):
        """
        Download the latest bulk company data from Companies House

        Args:
            year: Year (defaults to current year)
            month: Month (defaults to current month)

        Returns:
            Boolean indicating success
        """
        if year is None or month is None:
            now = datetime.now()
            year = now.year
            month = now.month

        # Format the URL
        date_str = f"{year}-{month:02d}-01"
        url = f"{self.base_url}/BasicCompanyDataAsOneFile-{date_str}.zip"

        print(f"Downloading from: {url}")
        print("This may take a few minutes (file is ~400MB)...")

        try:
            # Download the ZIP file
            response = requests.get(url, stream=True)
            response.raise_for_status()

            # Extract and read the CSV
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                csv_filename = z.namelist()[0]
                print(f"Extracting {csv_filename}...")

                with z.open(csv_filename) as f:
                    # Read CSV with proper encoding
                    self.data = pd.read_csv(f, encoding='utf-8', low_memory=False)

            print(f"Successfully loaded {len(self.data):,} companies!")
            return True

        except requests.exceptions.HTTPError as e:
            print(f"Error downloading file: {e}")
            print("The file for this month may not be available yet.")
            print("Try the previous month or check: http://download.companieshouse.gov.uk/en_output.html")
            return False
        except Exception as e:
            print(f"Error: {e}")
            return False

    def filter_by_town(self, town_name):
        """
        Filter companies by town in their registered address

        Args:
            town_name: Name of the town to filter by (case-insensitive)

        Returns:
            DataFrame of matching companies
        """
        if self.data is None:
            print("No data loaded. Please download bulk data first.")
            return None

        # The CSV has a 'RegAddress.PostTown' column for town
        town_col = 'RegAddress.PostTown'

        if town_col not in self.data.columns:
            print(f"Column {town_col} not found. Available columns:")
            print(self.data.columns.tolist())
            return None

        # Filter (case-insensitive)
        mask = self.data[town_col].str.contains(town_name, case=False, na=False)
        filtered = self.data[mask].copy()

        print(f"Found {len(filtered):,} companies in {town_name}")
        return filtered

    def filter_by_postcode_prefix(self, postcode_prefixes):
        """
        Filter companies by postcode prefix(es)

        Args:
            postcode_prefixes: String or list of strings with postcode prefix(es)
                              e.g. "BH9 " or ["BH9 ", "BH21 "]
                              Note: Include the space after the number if needed

        Returns:
            DataFrame of matching companies
        """
        if self.data is None:
            print("No data loaded. Please download bulk data first.")
            return None

        # The CSV has a 'RegAddress.PostCode' column
        postcode_col = 'RegAddress.PostCode'

        if postcode_col not in self.data.columns:
            print(f"Column {postcode_col} not found. Available columns:")
            print(self.data.columns.tolist())
            return None

        # Convert single string to list
        if isinstance(postcode_prefixes, str):
            postcode_prefixes = [postcode_prefixes]

        # Create a mask for each prefix and combine them with OR
        mask = pd.Series([False] * len(self.data))

        for prefix in postcode_prefixes:
            # Match postcodes that start with the prefix (case-insensitive)
            prefix_mask = self.data[postcode_col].str.upper().str.startswith(prefix.upper(), na=False)
            mask = mask | prefix_mask
            print(f"  - Found {prefix_mask.sum():,} companies with postcode starting '{prefix}'")

        filtered = self.data[mask].copy()

        print(f"\nTotal companies found: {len(filtered):,}")
        return filtered

    def get_company_summary(self, df):
        """
        Get a clean summary of company data

        Args:
            df: DataFrame of companies

        Returns:
            DataFrame with selected columns
        """
        # Select relevant columns
        columns_to_keep = [
            'CompanyName',
            'CompanyNumber',
            'CompanyCategory',
            'CompanyStatus',
            'RegAddress.AddressLine1',
            'RegAddress.AddressLine2',
            'RegAddress.PostTown',
            'RegAddress.County',
            'RegAddress.PostCode',
            'IncorporationDate',
            'SICCode.SicText_1'
        ]

        # Only keep columns that exist
        available_cols = [col for col in columns_to_keep if col in df.columns]

        return df[available_cols]

    def export_to_csv(self, df, filename):
        """
        Export filtered data to CSV

        Args:
            df: DataFrame to export
            filename: Output filename
        """
        df.to_csv(filename, index=False)
        print(f"Data exported to {filename}")


def main():
    """
    Example usage
    """
    # Initialize
    bulk = CompaniesHouseBulkData()

    # Download the latest bulk data
    print("=" * 60)
    print("STEP 1: Downloading Bulk Company Data")
    print("=" * 60)

    success = bulk.download_bulk_data()

    if not success:
        print("\nTrying previous month...")
        now = datetime.now()
        prev_month = now.month - 1 if now.month > 1 else 12
        prev_year = now.year if now.month > 1 else now.year - 1
        success = bulk.download_bulk_data(year=prev_year, month=prev_month)

    if not success:
        print("Could not download data. Please check the Companies House website.")
        return

    # Filter by postcode prefix
    print("\n" + "=" * 60)
    print("STEP 2: Filtering by Postcode Prefix")
    print("=" * 60)

    # SET YOUR POSTCODE PREFIXES HERE
    # You can use a single prefix or multiple prefixes
    postcode_prefixes = ["BH9 ", "BH21 "]  # Change these to your desired postcodes

    companies = bulk.filter_by_postcode_prefix(postcode_prefixes)

    if companies is not None and len(companies) > 0:
        # Get summary
        summary = bulk.get_company_summary(companies)

        # Display first 10 results
        print(f"\nFirst 10 companies:")
        print("-" * 60)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        print(summary.head(10))

        # Export to CSV
        print("\n" + "=" * 60)
        print("STEP 3: Exporting Results")
        print("=" * 60)

        # Create filename from postcode prefixes
        prefix_str = "_".join([p.strip().replace(" ", "") for p in postcode_prefixes])
        output_file = f"companies_postcodes_{prefix_str}.csv"
        bulk.export_to_csv(summary, output_file)

        # Show statistics
        print("\n" + "=" * 60)
        print("STATISTICS")
        print("=" * 60)
        print(f"Total companies found: {len(companies):,}")
        print(f"\nCompany Status breakdown:")
        print(companies['CompanyStatus'].value_counts())

        print(f"\nTop 5 Company Types:")
        print(companies['CompanyCategory'].value_counts().head())


if __name__ == "__main__":
    main()

STEP 1: Downloading Bulk Company Data
This may take a few minutes (file is ~400MB)...
Extracting BasicCompanyDataAsOneFile-2025-11-01.csv...
Successfully loaded 5,681,537 companies!

STEP 2: Filtering by Postcode Prefix
  - Found 1,741 companies with postcode starting 'BH9 '
  - Found 6,750 companies with postcode starting 'BH21 '

Total companies found: 8,491

First 10 companies:
------------------------------------------------------------
                                            CompanyName  \
223                                  & DAUGHTER LIMITED   
3523            1 PORTCHESTER PLACE RTM COMPANY LIMITED   
4126  1-12 ELIZABETH PLACE PROPERTY MANAGEMENT COMPA...   
6132                         101 BELLE VUE NO 1 LIMITED   
6560                            1024 CONSULTING LIMITED   
7566                109 TALBOT ROAD RTM COMPANY LIMITED   
8253                       11 GORDON HOUSE ROAD LIMITED   
8432                  11 OAKLEY HILL MANAGEMENT LIMITED   
8690         11 WINDSOR 